In [2]:
import glob
import ntpath
import os
import pickle

import cv2
import einops
import numpy as np
import scipy.special
import torch
import yaml
from fvcore.common.config import CfgNode as CN
from tqdm import tqdm

from data.ava_dataset import MultiCaptureDataset as AvaMultiCaptureDataset
from data.ava_dataset import none_collate_fn
from utils import get_autoencoder, load_checkpoint, render_img, train_csv_loader


In [3]:
def fetch_id_embedding(uid):
    with open(f"id_embeddings/{uid}.pickle", "rb") as f:
        id_embedding = pickle.load(f)
    return id_embedding


def id_cond_to_device(id_cond, device=torch.device("cuda")):
    # put id_cond on the gpu

    id_cond2 = {}
    id_cond2["z_tex"] = id_cond["z_tex"].detach().to(device)
    id_cond2["z_geo"] = id_cond["z_geo"].detach().to(device)
    id_cond2["b_tex"] = [None, None, None, None, None, None, None, None]
    id_cond2["b_geo"] = [None, None, None, None, None, None, None, None]
    for i in range(8):
        id_cond2["b_tex"][i] = id_cond["b_tex"][i].detach().to(device)
        id_cond2["b_geo"][i] = id_cond["b_geo"][i].detach().to(device)

    return id_cond2


def generate_image(ae, id_cond, cudadriver):
    id_cond = id_cond_to_device(id_cond)

    output = ae(
        camrot=cudadriver["camrot"],
        campos=cudadriver["campos"],
        focal=cudadriver["focal"],
        princpt=cudadriver["princpt"],
        modelmatrix=cudadriver["modelmatrix"],
        avgtex=cudadriver["avgtex"],
        verts=cudadriver["verts"],
        neut_avgtex=cudadriver["neut_avgtex"],
        neut_verts=cudadriver["neut_verts"],
        target_neut_avgtex=None,
        target_neut_verts=None,
        id_cond=id_cond,
        pixelcoords=cudadriver["pixelcoords"],
    )

    rgb = output["irgbrec"].detach().cpu().numpy()
    rgb = einops.rearrange(rgb, "1 c h w -> h w c")

    return rgb


def increase_brightness(img, value=30):
    img_start = np.copy(img)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = img_grey < 1
    mask = np.dstack([mask, mask, mask])

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    img[mask] = img_start[mask]

    final_img = (img_start / 255.0) / (1.004 - (img / 255.0))
    final_img = 0.5 * final_img + (0.5 * (img / 255.0))

    final_img[final_img > 1] = 1
    final_img = (final_img * 255).astype(np.uint8)

    return final_img


def render(ae, id_cond, cudadriver, out_path: str = "test.png"):
    rgb = generate_image(ae, id_cond, cudadriver)
    rgb = increase_brightness(rgb, value=30)
    render_img([[rgb]], out_path)


In [4]:
checkpoint = "aeparams_1440000.pt"  # the pretrained model
config = "configs/config.yaml"
opts = []


with open(config, "r") as file:
    config = CN(yaml.load(file, Loader=yaml.UnsafeLoader))

config.merge_from_list(opts)

train_params = config.train

# Train dataset mean/std texture and vertex for normalization
train_captures, train_dirs = train_csv_loader(
    train_params.dataset_dir, train_params.data_csv, train_params.nids
)
dataset = AvaMultiCaptureDataset(
    train_captures, train_dirs, downsample=train_params.downsample
)

batchsize = 1
numworkers = 1

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batchsize,
    shuffle=False,
    drop_last=True,
    num_workers=numworkers,
    collate_fn=none_collate_fn,
)

# Get Autoencoder
assetpath = "assets"
ae = get_autoencoder(dataset, assetpath=assetpath)
# Load from checkpoint
ae = load_checkpoint(ae, checkpoint).cuda()
# Set to Evaluation mode
ae.eval()

id_model = ae.id_encoder
texmean = dataset.texmean
vertmean = dataset.vertmean
texstd = dataset.texstd
vertstd = dataset.vertstd

# Delete dataset because it is no longer used
del dataset


user_ids = []

for ui in glob.glob("E://codec_dataset/*"):
    user_ids.append(ntpath.basename(ui))

Loading single id captures: 100%|██████████| 256/256 [01:37<00:00,  2.62it/s]


@@@ Get autoencoder ABLATION CONFIG FILE : length of data set : 256
dataset vertmean: (7306, 3)
id_encoder params: 5062060
encoder params: 5_551_232
decoder params: 35_918_504
colorcal params: 3_252
bgmodel params: 454_739
total params: 46_991_899


d:\github\ava-256\utils.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = th.load(filename)


In [5]:
tex_embeddings = []
geo_embeddings = []
for uid in tqdm(user_ids):
    id_embedding = fetch_id_embedding(uid)
    tex_embeddings.append(id_embedding["id_cond"]["z_tex"])
    geo_embeddings.append(id_embedding["id_cond"]["z_geo"])

100%|██████████| 256/256 [00:38<00:00,  6.65it/s]


In [29]:
avg_tex_bias = [[], [], [], [], [], [], [], []]
avg_geo_bias = [[], [], [], [], [], [], [], []]
for uid in tqdm(user_ids):
    id_embedding = fetch_id_embedding(uid)
    for i in range(8):
        avg_tex_bias[i].append(id_embedding["id_cond"]["b_tex"][i])
        avg_geo_bias[i].append(id_embedding["id_cond"]["b_geo"][i])

for i in range(8):
    avg_tex_bias[i] = torch.mean(torch.stack(avg_tex_bias[i]), dim=0)
    avg_geo_bias[i] = torch.mean(torch.stack(avg_geo_bias[i]), dim=0)
    print(avg_tex_bias[i].shape)


100%|██████████| 256/256 [03:44<00:00,  1.14it/s]


torch.Size([1, 256, 8, 8])
torch.Size([1, 128, 16, 16])
torch.Size([1, 128, 32, 32])
torch.Size([1, 64, 64, 64])
torch.Size([1, 64, 128, 128])
torch.Size([1, 32, 256, 256])
torch.Size([1, 16, 512, 512])
torch.Size([1, 3, 1024, 1024])


In [6]:
# # save these embeddings to a file


# tex_embeddings_numpy, geo_embeddings_numpy = [], []
# for i in range(len(tex_embeddings)):
#     tex_embeddings_numpy.append(tex_embeddings[i].cpu().detach().numpy())
#     geo_embeddings_numpy.append(geo_embeddings[i].cpu().detach().numpy())
# tex_embeddings_numpy = np.asarray(tex_embeddings_numpy)
# geo_embeddings_numpy = np.asarray(geo_embeddings_numpy)

# with open("tex_embeddings.pickle", "wb") as f:
#     pickle.dump(tex_embeddings_numpy, f)
# with open("geo_embeddings.pickle", "wb") as f:
#     pickle.dump(geo_embeddings_numpy, f)


In [ ]:
# PCA to 16 dims
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=16)
geo_pca = PCA(n_components=16)


tex_np = np.asarray(tex_embeddings).reshape(len(user_ids), -1)
scaler = StandardScaler()
tex_np = scaler.fit_transform(tex_np)
# tex_np = tex_np / np.linalg.norm(tex_np, axis=1, keepdims=True)

pca_embeddings = pca.fit_transform(tex_np)
# pca_embeddings = pca_embeddings / np.linalg.norm(pca_embeddings, axis=1, keepdims=True)


geo_np = np.asarray(geo_embeddings).reshape(len(user_ids), -1)
geo_np = scaler.fit_transform(geo_np)
geo_pca_embeddings = geo_pca.fit_transform(geo_np)


In [8]:
with open("pca_embeddings.pickle", "wb") as f:
    pickle.dump(np.asarray(pca_embeddings), f)


In [9]:
from anonymization import anonymize


def generate_anon_embedding(
    original_idx, pca_embeddings, epsilon: float = -1, theta: float = 0
):
    zs_tex, zs_geo = [], []
    bs_tex, bs_geo = [[] for _ in range(8)], [[] for _ in range(8)]

    A = anonymize(pca_embeddings[original_idx], epsilon=epsilon, theta=theta)

    dists = []
    for i in range(0, len(user_ids)):
        B = pca_embeddings[i]
        cos_sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
        dists.append(cos_sim)

    sorted_ids = [x for _, x in sorted(zip(dists, user_ids), key=lambda pair: pair[0])]
    dists = sorted(dists)

    top5_ids = sorted_ids[-8:]
    top5_dists = dists[-8:]
    top5_dists = scipy.special.softmax(np.asarray(top5_dists) * 8)

    for weight, uid in zip(top5_dists, top5_ids):
        id_embedding = fetch_id_embedding(uid)
        id_cond = id_embedding["id_cond"]

        zs_tex.append(weight * id_cond["z_tex"])
        zs_geo.append(weight * id_cond["z_geo"])
        for i in range(8):
            bs_tex[i].append(weight * id_cond["b_tex"][i])
            bs_geo[i].append(weight * id_cond["b_geo"][i])

    new_id_cond = {}
    new_id_cond["z_tex"] = torch.sum(torch.stack(zs_tex), dim=0)
    new_id_cond["z_geo"] = torch.sum(torch.stack(zs_geo), dim=0)
    new_id_cond["b_tex"] = []
    new_id_cond["b_geo"] = []

    for i in range(8):
        new_id_cond["b_tex"].append(torch.sum(torch.stack(bs_tex[i]), dim=0))
        new_id_cond["b_geo"].append(torch.sum(torch.stack(bs_geo[i]), dim=0))

    return new_id_cond

In [34]:
def generate_anon_embedding_alternative(
    original_idx,
    pca_embeddings,
    geo_pca_embeddings,
    epsilon: float = -1,
    theta: float = 0,
):
    zs_tex, zs_geo = [], []
    bs_tex, bs_geo = [[] for _ in range(8)], [[] for _ in range(8)]

    A = anonymize(pca_embeddings[original_idx], epsilon=epsilon, theta=theta)
    G = anonymize(geo_pca_embeddings[original_idx], epsilon=epsilon, theta=theta)

    dists = []
    for i in range(0, len(user_ids)):
        B = pca_embeddings[i]
        cos_sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
        dists.append(cos_sim)

    sorted_ids = [x for _, x in sorted(zip(dists, user_ids), key=lambda pair: pair[0])]
    dists = sorted(dists)

    top5_ids = sorted_ids[-8:]
    top5_dists = dists[-8:]
    top5_dists = scipy.special.softmax(np.asarray(top5_dists) * 8)

    for weight, uid in zip(top5_dists, top5_ids):
        id_embedding = fetch_id_embedding(uid)
        id_cond = id_embedding["id_cond"]

        # zs_tex.append(weight * id_cond["z_tex"])
        # zs_geo.append(weight * id_cond["z_geo"])
        for i in range(8):
            bs_tex[i].append(weight * id_cond["b_tex"][i])
            bs_geo[i].append(weight * id_cond["b_geo"][i])

    A = (A * np.std(pca_embeddings[original_idx])) + np.mean(
        pca_embeddings[original_idx]
    )
    G = (G * np.std(geo_pca_embeddings[original_idx])) + np.mean(
        geo_pca_embeddings[original_idx]
    )

    A = pca.inverse_transform(A)

    ref_cond = fetch_id_embedding(user_ids[original_idx])["id_cond"]

    A = torch.tensor(
        A.reshape(1, 16, 4, 4),
        dtype=ref_cond["z_tex"].dtype,
        device=ref_cond["z_tex"].device,
    )
    G = pca.inverse_transform(G)
    G = torch.tensor(
        G.reshape(1, 16, 4, 4),
        dtype=ref_cond["z_tex"].dtype,
        device=ref_cond["z_tex"].device,
    )

    new_id_cond = {}
    new_id_cond["z_tex"] = A
    new_id_cond["z_geo"] = G
    new_id_cond["b_tex"] = []
    new_id_cond["b_geo"] = []
    # new_id_cond["b_tex"] = avg_tex_bias
    # new_id_cond["b_geo"] = avg_geo_bias

    for i in range(8):
        new_id_cond["b_tex"].append(torch.sum(torch.stack(bs_tex[i]), dim=0))
        new_id_cond["b_geo"].append(torch.sum(torch.stack(bs_geo[i]), dim=0))

    return new_id_cond

In [35]:
# test alternate mechanism
for j in range(10):
    id_embedding = fetch_id_embedding(user_ids[j])
    front_driver = id_embedding["cudadriver"]

    img_front = generate_image(ae, id_embedding["id_cond"], front_driver)
    img_row = []

    print(f"test {j}")
    for theta in [0, 90, 180]:
        id_cond = generate_anon_embedding_alternative(
            j, pca_embeddings, geo_pca_embeddings, theta=theta
        )
        img_gen = generate_image(ae, id_cond, front_driver)
        img_gen = increase_brightness(img_gen)
        img_row.append(img_gen)

    render_img([img_row], f"out/pca_recon_{j}.png")


test 0
test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9


In [ ]:
# set up some visualization tests
for j in range(10):
    id_embedding = fetch_id_embedding(user_ids[j])
    front_driver = id_embedding["cudadriver"]

    img_front = generate_image(ae, id_embedding["id_cond"], front_driver)

    print(f"test {j}")
    for theta in [0, 30, 60, 90, 120]:
        id_cond = generate_anon_embedding(j, pca_embeddings, theta=theta)
        img_gen = generate_image(ae, id_cond, front_driver)

        render_img([[img_front, img_gen]], f"out/anon_{j}_theta{theta}.png")

    for eps in [0, 1, 10, 100, 1000]:
        id_cond = generate_anon_embedding(j, pca_embeddings, epsilon=eps)
        img_gen = generate_image(ae, id_cond, front_driver)

        render_img([[img_front, img_gen]], f"out/anon_{j}_eps{eps}.png")


test 0
test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9


In [ ]:
# Visualizing IdentityDP applied to these features
img_cols = []
for j in range(10):
    id_embedding = fetch_id_embedding(user_ids[j])
    front_driver = id_embedding["cudadriver"]

    id_cond = id_embedding["id_cond"]
    img_front = generate_image(ae, id_embedding["id_cond"], front_driver)
    img_row = [img_front]

    print(f"test {j}")
    sens = 2
    for mag in [1, 0.5, 0.2, 0.1]:
        noisy_id_cond = id_cond.copy()
        noisy_id_cond["z_tex"] = id_cond["z_tex"] + torch.tensor(
            np.random.laplace(0, sens / mag, id_cond["z_tex"].shape),
            dtype=id_cond["z_tex"].dtype,
            device=id_cond["z_tex"].device,
        )
        noisy_id_cond["z_geo"] = id_cond["z_geo"] + torch.tensor(
            np.random.laplace(0, sens / mag, id_cond["z_geo"].shape),
            dtype=id_cond["z_tex"].dtype,
            device=id_cond["z_tex"].device,
        )
        # for i in range(8):
        #    noisy_id_cond["b_tex"][i] = id_cond["b_tex"][i] + torch.tensor(np.random.laplace(0, sens / mag, id_cond["b_tex"][i].shape), dtype=id_cond["z_tex"].dtype, device=id_cond["z_tex"].device)
        #    noisy_id_cond["b_geo"][i] = id_cond["b_geo"][i] + torch.tensor(np.random.laplace(0, sens / mag, id_cond["b_geo"][i].shape), dtype=id_cond["z_tex"].dtype, device=id_cond["z_tex"].device)

        img_gen = generate_image(ae, noisy_id_cond, front_driver)
        img_row.append(img_gen)
        render_img([img_row], f"out/idp_test{j}.png")
    img_cols.append(img_row)


test 0
test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9


In [ ]:
# Visualizing rotation with no PCA applied to these features


def rot_feat(feat, eps, theta):
    dims = feat.shape
    feat = feat.view(1, -1)
    feat = torch.tensor(
        anonymize(feat.cpu().detach().numpy(), epsilon=eps, theta=theta),
        device=feat.device,
        dtype=feat.dtype,
    )
    # z_tex = torch.zeros_like(z_tex)
    feat = feat.view(dims)
    return feat


img_cols = []
for j in range(10):
    id_embedding = fetch_id_embedding(user_ids[j])
    front_driver = id_embedding["cudadriver"]

    id_cond = id_embedding["id_cond"]
    img_front = generate_image(ae, id_embedding["id_cond"], front_driver)
    img_row = [img_front]

    print(f"test {j}")
    eps = 0
    for theta in [10]:  # , 20, 30]:
        noisy_id_cond = id_cond.copy()
        noisy_id_cond["z_tex"] = rot_feat(noisy_id_cond["z_tex"], eps, theta)
        noisy_id_cond["z_geo"] = rot_feat(noisy_id_cond["z_geo"], eps, theta)

        # for i in range(1):
        #     try:
        #         noisy_id_cond["b_tex"][i] = rot_feat(noisy_id_cond["b_tex"][i], eps, theta)
        #         noisy_id_cond["b_geo"][i] = rot_feat(noisy_id_cond["b_geo"][i], eps, theta)
        #     except:
        #         print(f"failed on {i}")
        #         break

        img_gen = generate_image(ae, noisy_id_cond, front_driver)
        img_row.append(img_gen)
        render_img([img_row], f"out/no_pca_test{j}.png")
    img_cols.append(img_row)


test 0
test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9


In [11]:
trans_img_cols = [list(row) for row in zip(*img_cols)]
render_img(trans_img_cols, "out/no_pca_test_all.png")

In [36]:
for j, uid in tqdm(enumerate(user_ids)):
    id_embedding = fetch_id_embedding(uid)
    # # already done
    render(
        ae,
        id_embedding["id_cond"],
        id_embedding["cudadriver"],
        f"D://github/FaceAnonEval/Datasets/codec/avatars/{id_embedding['uid']}.png",
    )

    for theta in [30, 60, 90, 120, 150, 180]:
        os.makedirs(
            f"D://github/FaceAnonEval/Anonymized Datasets/codec_theta{theta}/avatars",
            exist_ok=True,
        )
        id_cond = generate_anon_embedding(j, pca_embeddings, theta=theta)
        render(
            ae,
            id_cond,
            id_embedding["cudadriver"],
            f"D://github/FaceAnonEval/Anonymized Datasets/codec_theta{theta}/avatars/{id_embedding['uid']}.png",
        )

    for eps in [-1, 0, 1, 10, 50, 100, 500, 1000]:
        os.makedirs(
            f"D://github/FaceAnonEval/Anonymized Datasets/codec_eps{eps}/avatars",
            exist_ok=True,
        )
        id_cond = generate_anon_embedding(j, pca_embeddings, epsilon=eps)
        render(
            ae,
            id_cond,
            id_embedding["cudadriver"],
            f"D://github/FaceAnonEval/Anonymized Datasets/codec_eps{eps}/avatars/{id_embedding['uid']}.png",
        )

256it [2:29:46, 35.10s/it]


In [ ]:
# ALTERNATE DATASET
os.makedirs(
    "D://github/FaceAnonEval/Datasets/codecpca/avatars",
    exist_ok=True,
)

for j, uid in tqdm(enumerate(user_ids)):
    id_embedding = fetch_id_embedding(uid)
    # already done
    render(
        ae,
        id_embedding["id_cond"],
        id_embedding["cudadriver"],
        f"D://github/FaceAnonEval/Datasets/codecpca/avatars/{id_embedding['uid']}.png",
    )

    for theta in [30, 60, 90, 120, 150, 180]:
        os.makedirs(
            f"D://github/FaceAnonEval/Anonymized Datasets/codecpca_theta{theta}/avatars",
            exist_ok=True,
        )
        id_cond = generate_anon_embedding_alternative(
            j, pca_embeddings, geo_pca_embeddings, theta=theta
        )
        render(
            ae,
            id_cond,
            id_embedding["cudadriver"],
            f"D://github/FaceAnonEval/Anonymized Datasets/codecpca_theta{theta}/avatars/{id_embedding['uid']}.png",
        )

    for eps in [-1, 0, 1, 10, 50, 100, 500, 1000]:
        os.makedirs(
            f"D://github/FaceAnonEval/Anonymized Datasets/codecpca_eps{eps}/avatars",
            exist_ok=True,
        )
        id_cond = generate_anon_embedding_alternative(
            j, pca_embeddings, geo_pca_embeddings, epsilon=eps
        )
        render(
            ae,
            id_cond,
            id_embedding["cudadriver"],
            f"D://github/FaceAnonEval/Anonymized Datasets/codecpca_eps{eps}/avatars/{id_embedding['uid']}.png",
        )

256it [3:00:41, 42.35s/it]


In [ ]:
print(user_ids)

['20230405--1635--AAN112', '20230831--0814--ADL311', '20230324--0820--AEY864', '20230810--1355--AJR151', '20230810--1630--ANX726', '20230629--1159--APP152', '20230726--1657--AYE877', '20230308--1352--BDF920', '20220412--1326--BGR645', '20220711--1300--BHC106', '20230316--1103--BHK376', '20220809--1034--BJM420', '20230328--0800--BLY735', '20220815--1307--BMP511', '20220627--0812--BPM833', '20220819--0803--BUS250', '20230914--1105--BXQ083', '20210818--1332--CDR970', '20220831--0751--CMS162', '20230224--1359--CMZ386', '20230309--0820--CPA784', '20230901--1429--CPP930', '20230728--0757--CRV122', '20230720--1025--CTP175', '20230706--1016--CVR434', '20230306--0812--CYQ521', '20230615--1702--DAT749', '20230404--1217--DED409', '20230908--1645--DHA971', '20220614--1135--DNM410', '20210819--0903--DOT682', '20220808--0809--DPE040', '20230207--1619--DTW301', '20230511--0810--DUA525', '20230817--1136--DZS003', '20230303--0802--EFQ599', '20211006--0836--EID363', '20230801--1420--EJG940', '20230410--

In [ ]:
# get the driver

from typing import Dict, Union

from data.ava_dataset import MultiCaptureDataset as AvaMultiCaptureDataset
from data.ava_dataset import SingleCaptureDataset as AvaSingleCaptureDataset
from data.utils import MugsyCapture
from utils import tocuda


def render_clip(target_idx, eps, theta, chosen_seg=None):
    target_id = user_ids[target_idx]

    os.makedirs(f"figures/{target_id}/eps{eps}_theta{theta}", exist_ok=True)

    if eps < 0 and theta <= 0:
        id_cond = fetch_id_embedding(target_id)["id_cond"]
    else:
        id_cond = generate_anon_embedding(
            target_idx, pca_embeddings, epsilon=eps, theta=theta
        )

    # Driver capture dataloader
    driver_capture = MugsyCapture(
        mcd=target_id.split("--")[0],
        mct=target_id.split("--")[1],
        sid=target_id.split("--")[2],
    )
    driver_dir = f"{train_params.dataset_dir}/{target_id}/decoder"
    driver_dataset = AvaSingleCaptureDataset(
        driver_capture, driver_dir, downsample=train_params.downsample
    )

    seg_ids = np.unique(driver_dataset.framelist["seg_id"])

    if chosen_seg is None:
        print(seg_ids)
        chosen_seg = seg_ids[0]

    if chosen_seg == "last":
        chosen_seg = seg_ids[-1]

    if chosen_seg == "SEN":
        driver_dataset.framelist = driver_dataset.framelist.loc[
            driver_dataset.framelist["seg_id"].str.contains("SEN")
        ]
    elif not chosen_seg == "long":
        # select only desired segments
        driver_dataset.framelist = driver_dataset.framelist.loc[
            driver_dataset.framelist["seg_id"] == chosen_seg
        ]
    else:
        driver_dataset.framelist = driver_dataset.framelist.iloc[0:250]

    if driver_dataset.framelist.values.tolist() == []:
        raise ValueError(
            f"Asked to render Segment {chosen_seg}, but there are no frames with that Segment in {driver_capture}"
        )

    if "401031" in driver_dataset.cameras:
        driver_dataset.cameras = ["401031"]
    elif "401880" in driver_dataset.cameras:
        driver_dataset.cameras = ["401880"]
    elif "401878" in driver_dataset.cameras:
        driver_dataset.cameras = ["401878"]
    else:
        driver_dataset.cameras = [driver_dataset.cameras[0]]

    driver_loader = torch.utils.data.DataLoader(
        driver_dataset,
        batch_size=batchsize,
        shuffle=False,
        drop_last=False,
        num_workers=numworkers,
        collate_fn=none_collate_fn,
    )

    it = 0

    with torch.no_grad():
        for driver in tqdm(driver_loader, desc="Rendering Frames"):
            # Skip if any of the frames is empty
            if driver is None:
                continue

            cudadriver: Dict[str, Union[torch.Tensor, int, str]] = tocuda(driver)

            img_gen = generate_image(ae, id_cond, cudadriver)
            img_gen = increase_brightness(img_gen, value=30)

            render_img(
                [[img_gen]], f"figures/{target_id}/eps{eps}_theta{theta}/{it}.png"
            )
            it += 1


In [30]:
for z in [4]:
    render_clip(z, eps=-1, theta=0, chosen_seg="SEN")

Rendering Frames: 100%|██████████| 756/756 [03:47<00:00,  3.33it/s]


In [26]:
# for z in range(10):
#     for eps in [20]:  # [1, 5, 10, 20, 50, 100]:
#         render_clip(z, eps=eps, theta=0, chosen_seg="long")

In [36]:
for z in [0, 4, 6, 7]:
    for theta in [45, 60, 70]:
        render_clip(z, eps=-1, theta=theta, chosen_seg="SEN")

Rendering Frames: 100%|██████████| 1206/1206 [06:14<00:00,  3.22it/s]


In [ ]:
from PIL import Image

tot_img = []

# for z, frame in zip([1, 9, 5, 8], [0, 19, 5, 10]):
for z, frame in zip([6, 5, 4, 8], [0, 19, 5, 10]):
    user_id = user_ids[z]

    inner_img = []
    for eps, theta in [
        # (1, 0),
        # (10, 0),
        # (50, 0),
        (-1, 0),
        (-1, 20),
        (-1, 40),
        (-1, 60),
        (-1, 80),
    ]:
        img = np.asarray(
            Image.open(f"figures/{user_id}/eps{eps}_theta{theta}/{frame}.png")
        )
        inner_img.append(img)
    tot_img.append(inner_img)

# if need to transpose: [list(i) for i in zip(*l)]
render_img(tot_img, "figures/avatar_results3.jpg")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\github\\ava-256\\figures\\20230726--1657--AYE877\\eps-1_theta20\\0.png'

In [38]:
from PIL import Image

try:
    for frame in tqdm(range(9999)):
        tot_img = []

        # for z in [1, 2, 4, 8]:  # vid2
        for z in [0, 4, 6, 7]:  # vid1
            user_id = user_ids[z]

            inner_img = []
            for eps, theta in [
                (-1, 0),
                (-1, 45),
                (-1, 60),
                (-1, 70),
            ]:
                img = np.asarray(
                    Image.open(f"figures/{user_id}/eps{eps}_theta{theta}/{frame}.png")
                )
                inner_img.append(img)
            tot_img.append(inner_img)

        # if need to transpose: [list(i) for i in zip(*l)]

        render_img(tot_img, f"figures/vid1/{frame}.jpg")
except Exception as e:
    print(e)
    print("a clip ended")

  6%|▌         | 556/9999 [00:50<14:09, 11.12it/s]

[Errno 2] No such file or directory: 'D:\\github\\ava-256\\figures\\20230405--1635--AAN112\\eps-1_theta0\\556.png'
a clip ended


In [ ]:
# render for figures


img_col = []
for j in range(10):
    id_embedding = fetch_id_embedding(user_ids[j])
    front_driver = id_embedding["cudadriver"]

    img_front = generate_image(ae, id_embedding["id_cond"], front_driver)
    img_row = [img_front]

    print(f"test {j}")
    for theta in [40, 50, 60, 70]:
        torch.manual_seed(0)
        np.random.seed(0)
        id_cond = generate_anon_embedding(j, pca_embeddings, theta=theta)
        img_gen = generate_image(ae, id_cond, front_driver)
        img_row.append(img_gen)
    img_col.append(img_row)

render_img(img_col, "gradual_change.jpg")


test 0
test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9
